## HALO position and attitude

### Data access
Some of the data, like the preliminary data of the HALO-(AC)<sup>3</sup> campaign, is stored on the (AC)<sup>3</sup> nextcloud server. This requires username and password as credentials ([registration](https://cloud.ac3-tr.de/index.php/login)) that need to be loaded from environment variables. 

In [1]:
import os
ac3cloud_username = os.environ['AC3_USER']
ac3cloud_password = os.environ['AC3_PASSWORD']

In [2]:
import ac3airborne

Show available data sets

In [3]:
cat = ac3airborne.get_intake_catalog()

In [4]:
list(cat['HALO-AC3']['HALO'])

['BAHAMAS',
 'BACARDI',
 'DROPSONDES',
 'GPS_INS',
 'HAMP_KV',
 'HAMP_11990',
 'HAMP_183',
 'HAMP_MIRA',
 'SMART']

Get the flight segments. We need to load an older version that includes the HALO-AC3 flights. The version labeled _latest_ only holds flights from campaigns pre HALO-AC3.

In [5]:
meta = ac3airborne.get_flight_segments()